<a href="https://colab.research.google.com/github/tomonari-masada/course2022-stats1/blob/main/binomial_ML_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 二項分布のパラメータの最尤推定をPyTorchで

* PyTorchのインポート

In [ ]:
import torch

## 二項分布の作り方を確認

* 総試行回数と、パラメータを指定。

* まずは$\phi=0$と設定して、100回試行の場合の52という回数の対数尤度を求める。
 * 尤度はほぼゼロになる。つまり、$\phi=0$のとき、52回という回数はほぼありえない、ということ。

In [ ]:
phi = torch.tensor([0.])
m = torch.distributions.binomial.Binomial(100, probs=phi)

In [ ]:
m.log_prob(torch.tensor([52]))

In [ ]:
m.log_prob(torch.tensor([52])).exp()

* 次に$\phi=0.5$と設定して、100回試行の場合の52という回数の対数尤度を求める。
 * 尤度は大きくなる。つまり、$\phi=0.5$のときなら、52回という回数はかなりありえる、ということ。

In [ ]:
phi = torch.tensor([0.5], requires_grad=True)
m = torch.distributions.binomial.Binomial(100, probs=phi)

In [ ]:
m.log_prob(torch.tensor([52]))

In [ ]:
m.log_prob(torch.tensor([52])).exp()

## 対数尤度最大化をPyTorchで実装

* 二項分布のパラメータは、微分可能なテンソルとして作っておく。
* そして、そのパラメータを更新するoptimizerを作る。
 * optimizerは、簡単にSGDにしておく。
 * 学習率は小さくしておかないと、最適化がうまくいかないかも。

In [ ]:
phi = torch.tensor([0.132], requires_grad=True) # 適当に初期化
optimizer = torch.optim.SGD([phi], lr=0.001)

* 1000回のイテレーションで、negative log likelihoodを最小化する。
 * PyTorchでは最小化の計算しかできないので、最大化したいときは、マイナスを付けたものを最小化する。

In [ ]:
for i in range(30):
  optimizer.zero_grad()
  m = torch.distributions.binomial.Binomial(100, probs=phi)
  loss = - m.log_prob(torch.tensor([52]))
  loss.backward()
  optimizer.step()
  print(f"{i+1} | {phi.item():.4f}")